In [18]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection string
engine = create_engine('postgresql://uvg_user:uvg_password@db:5432/health_data')

# Simple query to test the foundation layer
try:
    df = pd.read_sql("SELECT 1 as connection_status", engine)
    print("Connection Successful! Your Biomedical Data Stack is ready.")
    print(df)
except Exception as e:
    print(f"Connection Failed: {e}")

Connection Successful! Your Biomedical Data Stack is ready.
   connection_status
0                  1


In [24]:
from sqlalchemy import text

with engine.connect() as conn:
    db = conn.execute(text("SELECT current_database();")).fetchall()
    schema = conn.execute(text("SELECT current_schema();")).fetchall()

db, schema


([('health_data',)], [('public',)])

In [28]:
with engine.connect() as conn:
    pd.read_sql(
        text("""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = 'public';
        """),
        conn
    )

In [31]:
from sqlalchemy import text

schema_sql = """
CREATE TABLE IF NOT EXISTS patients (
  subject_id SERIAL PRIMARY KEY,
  external_id TEXT UNIQUE,
  full_name TEXT,
  sex CHAR(1),
  date_of_birth DATE
);

CREATE TABLE IF NOT EXISTS admissions (
  hadm_id SERIAL PRIMARY KEY,
  subject_id INT REFERENCES patients(subject_id),
  admittime TIMESTAMP,
  dischtime TIMESTAMP,
  admission_type TEXT
);

CREATE TABLE IF NOT EXISTS diagnoses (
  diagnosis_id SERIAL PRIMARY KEY,
  hadm_id INT REFERENCES admissions(hadm_id),
  diagnosis_text TEXT
);
"""

with engine.connect() as conn:
    conn.execute(text(schema_sql))
    conn.commit()


In [32]:
with engine.connect() as conn:
    pd.read_sql(
        text("""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = 'public';
        """),
        conn
    )


In [33]:
pd.read_sql(
    text("SELECT * FROM patients ORDER BY external_id;"),
    engine
)

,subject_id,external_id,full_name,sex,date_of_birth,date_of_death
0,1,MRN-9001,Paciente Sin Fecha,F,None,None
1,2,MRN-9002,None,M,1990-01-01,None
2,3,MRN-9003,Paciente Ciudad Typo,M,1985-09-10,None


In [35]:
from sqlalchemy import text

pd.read_sql(
    text("""
    SELECT subject_id, external_id, full_name, sex, date_of_birth
    FROM patients
    WHERE external_id LIKE 'MRN-9%'
    ORDER BY external_id;
    """),
    engine
)

,subject_id,external_id,full_name,sex,date_of_birth
0,1,MRN-9001,Paciente Sin Fecha,F,None
1,2,MRN-9002,None,M,1990-01-01
2,3,MRN-9003,Paciente Ciudad Typo,M,1985-09-10


In [36]:
pd.read_sql(
    text("""
    SELECT COUNT(*) AS n_missing_name
    FROM patients
    WHERE full_name IS NULL;
    """),
    engine
)

,n_missing_name
0,1


In [37]:
pd.read_sql(
    text("""
    SELECT d.diagnosis_id,
           d.diagnosis_text
    FROM diagnoses d
    WHERE d.diagnosis_text LIKE '%Guate%';
    """),
    engine
)


,diagnosis_id,diagnosis_text
0,1,Guateeeemala referral note
